In [1]:
import ray 
import Levenshtein
import pandas as pd 
import numpy as np 

In [2]:
if ray.is_initialized():
    ray.shutdown()
ray.init()

2024-07-10 08:37:05,807	INFO worker.py:1762 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Python version:,3.11.7
Ray version:,2.31.0
Dashboard:,http://127.0.0.1:8265


In [3]:
ray.cluster_resources()

{'node:192.168.4.4': 1.0,
 'node:__internal_head__': 1.0,
 'CPU': 8.0,
 'memory': 10726979175.0,
 'object_store_memory': 5363489587.0,
 'accelerator_type:T4': 1.0,
 'GPU': 1.0}

In [4]:
import os 
root_path='/home/ubuntu/acoustic_stuff/hindi-acoustic-word-embedding/dataset/train_aligned_dataset'
csv_file=os.path.join(root_path,'train_reduced_data.csv')
ds=ray.data.read_csv(csv_file)


In [5]:
df=ds.to_pandas()

2024-07-10 08:37:39,967	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2024-07-10_08-37-02_160065_600347/logs/ray-data
2024-07-10 08:37:39,968	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadCSV]


- ReadCSV->SplitBlocks(16) 1:   0%|          | 0/1 [00:00<?, ?it/s]

Running 0:   0%|          | 0/1 [00:00<?, ?it/s]

In [34]:
import random

#@ray.remote
def sample_different_transcript(row,all_transcripts,audio_dict):
    original_transcript = row[1][3]
    other_transcripts = [t for t in all_transcripts if t != original_transcript]
    new_transcript = random.choice(other_transcripts)
    audio_path=random.choice(audio_dict[new_transcript])
    return (new_transcript,audio_path)

In [21]:
audio_dict = df.groupby('transcript')['audio_path'].apply(list).to_dict()


In [18]:
for row in sample_df.iterrows():
    print(row[1][3])
    break

यूराल


/tmp/ipykernel_600347/2520510720.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(row[1][3])


In [35]:
import time 
all_transcripts = df['transcript'].unique().tolist()
audio_dict = df.groupby('transcript')['audio_path'].apply(list).to_dict()
#sample_df=df.head(1000)
#test_df=df.head(2000)

start=time.time()
#row_objects = [sample_different_transcript.remote(row, all_transcripts=all_transcripts,audio_dict=audio_dict) for _, row in sample_df.iterrows()]
row_objects=[sample_different_transcript(row,all_transcripts=all_transcripts,audio_dict=audio_dict) for row in df.iterrows()]
end=time.time()

print(end-start)
#results = ray.get(row_objects)

/tmp/ipykernel_600347/1879518816.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  original_transcript = row[1][3]


41.375341176986694


In [36]:
df['negative_samples']=row_objects

In [42]:
df['negative_samples'][0][1]

'844424931008282-142-m_seg_3.wav'

In [45]:
dir_name=os.path.dirname(csv_file)
audio_path=os.path.join(dir_name,'844424931008282-142-m_seg_3.wav')

In [46]:
import IPython
import IPython.display
IPython.display.Audio(audio_path,rate=8000)

In [ ]:
# Create a new DataFrame with the results
new_df = pd.DataFrame(results, columns=['audio_path', 'new_transcript'])

# Shutdown Ray
ray.shutdown()

# Save the new DataFrame
new_df.to_csv('sampled_dataset.csv', index=False)

In [47]:
def sample_negative_transcript(curr_transcript,all_transcripts,audio_dict):
    other_transcripts = [t for t in all_transcripts if t != curr_transcript]
    new_transcript = random.choice(other_transcripts)
    audio_path=random.choice(audio_dict[new_transcript])
    return (new_transcript,audio_path)


def sample_negative(csv_file):
    if isinstance(csv_file,str):
        df=pd.read_csv(csv_file)
    else:
        df=csv_file
    
    transcripts = df['transcript'].unique().tolist()
    audio_dict = df.groupby('transcript')['audio_path'].apply(list).to_dict()
    curr_transcripts=df['transcript'].tolist()

    negatives=[sample_negative_transcript(cur_transcript,transcripts,audio_dict) for cur_transcript in curr_transcripts]

    df['negative_samples']=negatives

    df.to_csv(csv_file)

    print("Sampling Complete")
    


In [48]:
sample_negative(csv_file)

Sampling Complete


In [49]:
df=pd.read_csv(csv_file)

In [50]:
df.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,audio_path,transcript,duration,negative_samples
0,0,912,1062,844424933477212-934-m_seg_5.wav,यूराल,3.367-3.848 sec,"('ठिकाने', '844424931109777-458-f_seg_4.wav')"
1,1,36991,42694,844424931090773-590-f_seg_0.wav,साहित्य,0.000-1.452 sec,"('सौ', '844424933459745-558-m_seg_2.wav')"
2,2,65036,73828,844424930995811-736-f_seg_4.wav,भोरंज,2.336-2.859 sec,"('कहनेवाला', '844424930949679-736-f_seg_5.wav')"
3,3,13011,15176,844424930673861-252-f_seg_0.wav,अग्रोहा,0.000-1.283 sec,"('परिजनों', '844424932430432-384-f_seg_4.wav')"
4,4,61260,69715,844424933463461-572-m_seg_3.wav,राष्ट्रीय,2.727-3.650 sec,"('येरवडा', '844424933545620-1179-m_seg_5.wav')"
